In [ ]:
from pathlib import Path
from glob import glob
import warnings
import numpy as np
from sherpa.astro import ui
from sherpa.astro import datastack as ds
import sherpa

from astropy.io import fits
from astropy.table import Table
import astropy.units as u

import ChiantiPy.core as ch

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
obsids = ['5', 'XMM', '6443', '7435', '7436', '7437', '7438', '13250']
dslist = {}

In [ ]:
tgpart = ['dummy', 'heg', 'meg', 'leg']

def loadpha2(obsid):
    stack = ds.DataStack()
    dirname = glob(f'data/Chandra/tgcat/*_{obsid}*/')[0]
    pha2 = glob(dirname + '*pha2.*')
    ui.load_data(pha2[0])
    tab = Table.read(pha2[0], hdu=1, format='fits')
    # Use only 1 orders
    ind = np.abs(tab['TG_M']) == 1
    for row in tab[ind]:
        dataname = obsid + '_' + tgpart[row['TG_PART']] + '_' + str(row['TG_M'])
        ui.copy_data(row['SPEC_NUM'], dataname)
        m = row['TG_M']
        part = tgpart[row['TG_PART']]
        if ui.get_data().header['INSTRUME'] == 'ACIS':
            ui.load_arf(dataname, f'{dirname}{part}_{m}.arf.gz')
            ui.load_rmf(dataname, f'{dirname}{part}_{m}.rmf.gz') 
        else:  # HRC has no order-sorting
            sign = '' if m > 0 else '-'
            for num in [1,2,3]:
                ui.load_arf(dataname, f'{dirname}{part}_{sign}{num}.arf.gz', num)
                ui.load_rmf(dataname, f'{dirname}{part}_{sign}{num}.rmf.gz', num) 
        stack._add_dataset(dataname)
    # We copied all dataids to be used to new names, so delete the automatic read-in numbers
    for row in range(len(tab)):
        ui.delete_data(row + 1)
    return stack

In [ ]:
# Load in order of obervations, such that dslist is sorted by time
# First Chandra
dslist['5'] = loadpha2('5')

# Then XMM
path = 'data/XMM/0112880201/pps/'
ui.load_data('XMM_R1', path + 'P0112880201R1S004SRSPEC1003.FTZ')
ui.load_data('XMM_R2', path + 'P0112880201R2S005SRSPEC1003.FTZ')
ui.load_rmf('XMM_R1', path + 'P0112880201R1S004RSPMAT1003.FTZ')
ui.load_rmf('XMM_R2', path + 'P0112880201R2S005RSPMAT1003.FTZ')
stack = ds.DataStack()
stack._add_dataset('XMM_R1')
stack._add_dataset('XMM_R2')
dslist['XMM'] = stack

# and then all the other Chandra
for o in obsids[2:]:
    dslist[o] = loadpha2(o)

In [ ]:
# Some default settings
ui.set_analysis('wave')
ui.set_stat('cash')

In [ ]:
def indep_lines_model(lines, prefix=''):
    our_model = ui.const1d(name=f'{prefix}_bkg')
    our_model.c0.min = 0  # Fit goes heywire if we ever get negative values in model with cstat or cash
    for i, l in enumerate(lines):
        line = ui.delta1d(name=f'{prefix}_{i}')
        line.pos = l
        line.pos.frozen = True
        line.ampl = 1e-4
        line.ampl.min = 0
        our_model = our_model + line
    return our_model


In [ ]:
# the colors part is only needed for plot_fit etc. where more than one line is plotted
# per dataset
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def stack_plot(stack, func=ui.plot_data):
    for i, dataset in enumerate(stack.filter_datasets()):
        func(dataset['id'], overplot= i!= 0, color=colors[i])
        
    ax = plt.gca()
    ax.legend([dataset['id'].split('_', maxsplit=1)[1] for dataset in stack.filter_datasets()])
    return ax

In [ ]:
# I have a feeling this could be made more generically
# but for now just have them as separate functions

def fit(wvl, xmm=['XMM_R2']):
    for obsid, ds in dslist.items():
        ds.group_counts(1)
        ds.ignore(None, wvl[0])
        ds.ignore(wvl[1], None)
        ds.notice(*wvl)
        ui.set_stat("chi2gehrels")
        ui.set_method("levmar")
        if obsid == 'XMM':
            ui.fit(*xmm)
            ui.set_stat("cash")
            ui.fit(*xmm)
        else:
            ds.fit() 
            ui.set_stat("cash")
            ds.fit()
            
def conf(wvl, xmm=['XMM_R2']):
    conf_res = {}
    for obsid, ds in dslist.items():
        ds.group_counts(1)        
        ds.ignore(None, wvl[0])
        ds.ignore(wvl[1], None)
        ds.notice(*wvl)
        if obsid == 'XMM':
            ui.conf(*xmm)
        else:
            ds.conf()
        conf_res[obsid] = ui.get_conf_results()
    return conf_res

def plot(wvl, xmm=['XMM_R2'], group_counts=5):
    # Ignore a lot of "plotted errors on valid in this statistic errors"
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for obsid, ds in dslist.items():
            plt.figure()
            ds.group_counts(group_counts)
            ui.ignore(None, wvl[0])
            ui.ignore(wvl[1], None)
            ui.notice(*wvl)
            if obsid == 'XMM':
                for i, x in enumerate(xmm):
                    ui.plot_fit(x, color=colors[i], overplot= i != 0)
                ax = plt.gca()
                ax.legend(xmm)
            else:
                ax = stack_plot(ds, func=ui.plot_fit)
            ax.set_title(obsid)
            ax.set_xlim(*wvl)

## Ne IX
### He-like triplet

In [ ]:
for obsid, ds in dslist.items():
    ds.set_source(indep_lines_model([13.447, 13.553, 13.699], prefix=obsid))

In [ ]:
fit([13.40, 13.75], ['XMM_R2'])

In [ ]:
plot([13.40, 13.75], ['XMM_R2'])

In [ ]:
conf_res = conf([13.40, 13.75], ['XMM_R2'])

In [ ]:
farr = np.stack([np.array([c.parvals, c.parmins, c.parmaxes], 
                          dtype=np.floating) for c in conf_res.values()])

### He $\beta$

In [ ]:
for obsid, ds in dslist.items():
    ds.set_source(indep_lines_model([11.5467], prefix=obsid))

In [ ]:
fit([11.45, 11.65], ['XMM_R2'])

In [ ]:
plot([11.45, 11.65], ['XMM_R2'], group_counts=2)

In [ ]:
conf_res_hea = conf([11.4, 11.7], ['XMM_R2'])

In [ ]:
farr2 = np.stack([np.array([c.parvals, c.parmins, c.parmaxes], 
                          dtype=np.floating) for c in conf_res_hea.values()])

farr2.shape

In [ ]:
alpha2beta = farr[:, 0, 1] / farr2[:, 0, 1]
sig_alpha2beta = np.sqrt((farr[:, 2, 1] / farr2[:, 0, 1])**2 + (farr[:, 0, 1] * farr2[:, 2, 1] / farr2[:, 0, 1]**2)**2)

In [ ]:
plt.errorbar(obsids, alpha2beta, yerr=sig_alpha2beta, fmt='o')

In [ ]:
f2i = farr[:, 0, 3] / farr[:, 0, 2]
sig_f2i = np.sqrt((farr[:, 2, 3] / farr[:, 0, 2])**2 + (farr[:, 0, 3] * farr[:, 2, 2] / farr[:, 0, 2]**2)**2)

In [ ]:
fi2r = (farr[:, 0, 3] + farr[:, 0, 2]) / farr[:, 0, 1]
sig_fi2r = np.sqrt((farr[:, 2, 3] / farr[:, 0, 1])**2 + 
                   (farr[:, 2, 2] / farr[:, 0, 1])**2 + 
                   ((farr[:, 0, 3] + farr[:, 0, 2]) * farr[:, 2, 1] / farr[:, 0, 1]**2)**2)

In [ ]:
plt.errorbar(obsids, f2i, yerr=sig_f2i, fmt='o')

In [ ]:
plt.errorbar(obsids, fi2r, yerr=sig_fi2r, fmt='o')

In [ ]:
phabs = ui.xsphabs("phabs")
phabs.nH = 1
lam = np.arange(25., 1., -.1)
en = 12.4/lam
phabs.nH = 1
plt.plot(lam, phabs(en), label='$1 \\times 10^{22}$')
phabs.nH = .3
plt.plot(lam, phabs(en), label='$3\\times 10^{21}$')
phabs.nH = .1
plt.plot(lam, phabs(en), label='$1 \\times 10^{21}$')

plt.legend()

In [ ]:
logtemp = np.arange(6.1, 7.01, .1)
temp = 10**logtemp
ne9 = ch.ion('ne_9', temperature=temp, eDensity=1.e+9, em=1.e+27)
ne9.intensity()

In [ ]:
def lineemiss(ion, lvl1, lvl2):
    ind = (ion.Emiss[lvl1[0]] == lvl1[1]) & (ion.Emiss[lvl2[0]] == lvl2[1])
    # Do I want an assert statemnt here ot see that I selected the right lines?
    
    #assert ind.sum() == 1
    return ion.Emiss['emiss'][ind].flatten()

In [ ]:
e_heb = lineemiss(ne9, ('lvl1', 1), ('pretty2', '1s.3p 1P1.0'))
e_hea = lineemiss(ne9, ('lvl1', 1), ('pretty2', '1s.2p 1P1.0'))
e_i1 = lineemiss(ne9, ('lvl1', 1), ('pretty2', '1s.2p 3P2.0'))
e_i2 = lineemiss(ne9, ('lvl1', 1), ('pretty2', '1s.2p 2P1.0'))
e_f = lineemiss(ne9, ('lvl1', 1), ('pretty2', '1s.2s 3S1.0'))

In [ ]:
plt.plot(logtemp, e_hea / e_heb)
plt.xlabel('log T')
plt.ylabel('ratio Ly$\\alpha$/Ly$\\beta$')

In [ ]:
plt.plot(logtemp, (e_i1 + e_i2 + e_f) / e_hea)
plt.xlabel('log T')
plt.ylabel('ratio (f+i)/r')

In [ ]:
xp = (e_i1 + e_i2 + e_f) / e_hea
sortind = np.argsort(xp)
xp = xp[sortind]
fp = logtemp[sortind]
t_from_g = np.interp(fi2r, xp, fp)
t_from_g_up = np.interp(fi2r + sig_fi2r, xp, fp) - t_from_g
t_from_g_do = np.interp(fi2r - sig_fi2r, xp, fp) - t_from_g

In [ ]:
phabs = ui.xsphabs("phabs")
phabs.nH = .1

In [ ]:
# Input is "edge of bins", return values are for bin center.
# So need to make a few narrow bins around the range I care.
absval = phabs(([13.46, 13.45, 13.44, 11.56, 11.55, 11.54, 5.]
                * u.Angstrom).to(u.keV, equivalencies=u.spectral()).value)
absval

In [ ]:
abscoeffalpha = - 1e-21 * np.log(absval[2])
abscoeffbeta = - 1e-21 * np.log(absval[4])
abscoeffalpha, abscoeffbeta

In [ ]:
Gratio, Habobs = np.mgrid[.3:1.4:.01, 3:14:.01]

In [ ]:
logT_from_G = np.interp(Gratio, xp, fp)
Hab_from_logT = np.interp(logT_from_G, logtemp, e_hea / e_heb)

N_H = np.log(Habobs/Hab_from_logT) / (abscoeffbeta - abscoeffalpha) / 1e21

In [ ]:
# Define colors so that they match figure 3 in Brickhouse et al. (2012)
# Other colors are taken from default matplotlib color cycle
obscolors = {'5': '#ff7f0e', 
             'XMM': '#9467bd',
             '6443': '#8c564b', 
             '7435': '#d62728',
             '7436': '#1f77b4', 
             '7437': '#2ca02c',
             '7438': '#e377c2',
             '13250': '#7f7f7f'}

def grat2logt(g):
    return np.interp(g, xp, fp)

def logt2grat(logt):
    return np.interp(logt, fp[::-1], xp[::-1])

fig, ax = plt.subplots(figsize=(4,3))
cs = ax.contourf(Gratio, Habobs, np.ma.masked_less_equal(N_H, 0), cmap='binary')
cslines = ax.contour(Gratio, Habobs, N_H, levels=[0], linestyles=['dotted'], colors=['k'], linewidths=[4])
#ax.clabel(cs, cs.levels, inline=True, fontsize=10)
cb = fig.colorbar(cs, ax=ax)
for i, o in enumerate(obsids):
    eb = ax.errorbar(fi2r[i], alpha2beta[i] * 11.55 / 13.45, xerr=sig_fi2r[i], yerr=sig_alpha2beta[i],
                     fmt='o', label=o, color=obscolors[o])

secax = ax.secondary_xaxis('top', functions=(grat2logt, logt2grat))
secax.set_xlabel('Temperature [log T in K]')
    
ax.set_xlim(.4, 1.22)
ax.set_ylim(3, 13)
cb.set_label('$N_H$ [$10^{21}$ cm$^{-2}$]')
ax.set_xlabel('Observed line ratio $(f+i)/r$')
ax.set_ylabel('Observed line ratio He$\\alpha$/He$\\beta$')
ax.legend(loc='upper left', bbox_to_anchor=(1.3, 1))

fig.savefig('../plots/Ne-var.png', bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))

for i, o in enumerate(obsids):
    eb = ax.errorbar(fi2r[i], f2i[i], xerr=sig_fi2r[i], yerr=sig_f2i[i],
                     fmt='o', label=o, color=obscolors[o])

secax = ax.secondary_xaxis('top', functions=(grat2logt, logt2grat))
secax.set_xlabel('Temperature [log T in K]')
    
ax.set_xlim(.4, 1.22)
#ax.set_ylim(3, 13)
ax.set_xlabel('Observed line ratio $(f+i)/r$')
ax.set_ylabel('Observed line ratio $f/i$')
ax.legend(loc='upper left', bbox_to_anchor=(1.3, 1))

fig.savefig('../plots/Ne-var2.png', bbox_inches='tight', dpi=300)

## O VII

In [ ]:
for obsid, ds in dslist.items():
    ds.set_source(indep_lines_model([21.602, 21.804, 22.101], prefix=obsid))

In [ ]:
fit([21.4, 22.3], ['XMM_R1'])
conf_res = conf([21.4, 22.3], ['XMM_R1'])

In [ ]:
plot([21.4, 22.3], ['XMM_R1'])

In [ ]:
farr = np.stack([np.array([c.parvals, c.parmins, c.parmaxes], 
                          dtype=np.floating) for c in conf_res.values()])

### He b

In [ ]:
for obsid, ds in dslist.items():
    ds.set_source(indep_lines_model([18.627], prefix=obsid))

In [ ]:
fit([18.45, 18.80], ['XMM_R1'])
conf_res_heb = conf([18.45, 18.80], ['XMM_R1'])

In [ ]:
plot([18.45, 18.80], ['XMM_R1'])

In [ ]:
farr2 = np.stack([np.array([c.parvals, c.parmins, c.parmaxes], 
                          dtype=np.floating) for c in conf_res_heb.values()])

farr2.shape

In [ ]:
alpha2beta = farr[:, 0, 1] / farr2[:, 0, 1]
sig_alpha2beta = np.sqrt((farr[:, 2, 1] / farr2[:, 0, 1])**2 + (farr[:, 0, 1] * farr2[:, 2, 1] / farr2[:, 0, 1]**2)**2)

In [ ]:
plt.errorbar(obsids, alpha2beta, yerr=sig_alpha2beta, fmt='o')

In [ ]:
f2i = farr[:, 0, 3] / farr[:, 0, 2]
sig_f2i = np.sqrt((farr[:, 2, 3] / farr[:, 0, 2])**2 + (farr[:, 0, 3] * farr[:, 2, 2] / farr[:, 0, 2]**2)**2)

In [ ]:
fi2r = (farr[:, 0, 3] + farr[:, 0, 2]) / farr[:, 0, 1]
sig_fi2r = np.sqrt((farr[:, 2, 3] / farr[:, 0, 1])**2 + 
                   (farr[:, 2, 2] / farr[:, 0, 1])**2 + 
                   ((farr[:, 0, 3] + farr[:, 0, 2]) * farr[:, 2, 1] / farr[:, 0, 1]**2)**2)

In [ ]:
plt.errorbar(obsids, f2i, yerr=sig_f2i, fmt='o')

In [ ]:
plt.errorbar(obsids, fi2r, yerr=sig_fi2r, fmt='o')

In [ ]:
logtemp = np.arange(5.5, 7.01, .1)
temp = 10**logtemp
o7 = ch.ion('o_7', temperature=temp, eDensity=1.e+9, em=1.e+27)
o7.intensity()

In [ ]:
e_heb = lineemiss(o7, ('lvl1', 1), ('pretty2', '1s.3p 1P1.0'))
e_hea = lineemiss(o7, ('lvl1', 1), ('pretty2', '1s.2p 1P1.0'))
e_i1 = lineemiss(o7, ('lvl1', 1), ('pretty2', '1s.2p 3P2.0'))
#e_i2 = lineemiss(o7, ('lvl1', 1), ('pretty2', '1s.2p 2P1.0')) not in CHIATNI?!?
e_f = lineemiss(o7, ('lvl1', 1), ('pretty2', '1s.2s 3S1.0'))

In [ ]:
plt.plot(logtemp, e_hea / e_heb)
plt.xlabel('log T')
plt.ylabel('ratio Ly$\\alpha$/Ly$\\beta$')

In [ ]:
plt.plot(logtemp, (e_i1 + e_f) / e_hea)
plt.xlabel('log T')
plt.ylabel('ratio (f+i)/r')

In [ ]:
xp = (e_i1 + e_f) / e_hea
sortind = np.argsort(xp)
xp = xp[sortind]
fp = logtemp[sortind]
t_from_g = np.interp(fi2r, xp, fp)
t_from_g_up = np.interp(fi2r + sig_fi2r, xp, fp) - t_from_g
t_from_g_do = np.interp(fi2r - sig_fi2r, xp, fp) - t_from_g

In [ ]:
phabs = ui.xsphabs("phabs")
phabs.nH = .1

In [ ]:
# Input is "edge of bins", return values are for bin center.
# So need to make a few narrow bins around the range I care.
absval = phabs(([21.7, 21.6, 21.5, 18.7, 18.6, 18.5, 5.]
                * u.Angstrom).to(u.keV, equivalencies=u.spectral()).value)
absval

In [ ]:
abscoeffalpha = - 1e-21 * np.log(absval[2])
abscoeffbeta = - 1e-21 * np.log(absval[4])
abscoeffalpha, abscoeffbeta

In [ ]:
Gratio, Habobs = np.mgrid[.3:2:.01, 1:20:.01]

In [ ]:
logT_from_G = np.interp(Gratio, xp, fp)
Hab_from_logT = np.interp(logT_from_G, logtemp, e_hea / e_heb)

N_H = np.log(Habobs/Hab_from_logT) / (abscoeffbeta - abscoeffalpha) / 1e21

In [ ]:
# Define colors so that they match figure 3 in Brickhouse et al. (2012)
# Other colors are taken from default matplotlib color cycle
obscolors = {'5': '#ff7f0e', 
             'XMM': '#9467bd',
             '6443': '#8c564b', 
             '7435': '#d62728',
             '7436': '#1f77b4', 
             '7437': '#2ca02c',
             '7438': '#e377c2',
             '13250': '#7f7f7f'}

def grat2logt(g):
    return np.interp(g, xp, fp)

def logt2grat(logt):
    return np.interp(logt, fp[::-1], xp[::-1])

fig, ax = plt.subplots(figsize=(4,3))
cs = ax.contourf(Gratio, Habobs, np.ma.masked_less_equal(N_H, 0), cmap='binary')
cslines = ax.contour(Gratio, Habobs, N_H, levels=[0], linestyles=['dotted'], 
                     colors=['k'], linewidths=[4])
#ax.clabel(cs, cs.levels, inline=True, fontsize=10)
cb = fig.colorbar(cs, ax=ax)
for i, o in enumerate(obsids):
    eb = ax.errorbar(fi2r[i], alpha2beta[i] * 18.6 / 21.6, xerr=sig_fi2r[i], yerr=sig_alpha2beta[i],
                     fmt='o', label=o, color=obscolors[o])

secax = ax.secondary_xaxis('top', functions=(grat2logt, logt2grat))
secax.set_xlabel('Temperature [log T in K]')
    
ax.set_xlim(.4, 1.5)
ax.set_ylim(2, 11)
cb.set_label('$N_H$ [$10^{21}$ cm$^{-2}$]')
ax.set_xlabel('Observed line ratio $(f+i)/r$')
ax.set_ylabel('Observed line ratio He$\\alpha$/He$\\beta$')
ax.legend(loc='upper left', bbox_to_anchor=(1.3, 1))

fig.savefig('../plots/O-var.png', bbox_inches='tight', dpi=300)

# Notes
- XMM coords for R1 must be wrong - line is offset too far - > reduce RGS data by hand
- Chandra HEGT/HRC-I data is missing
- O VII and code is a dublication on Ne IX: Can I refactor more?
- Should I decouple fitting an ananylsis, similar to Dave etc al.? Put fits in scripts, run in eparate dirs to allow hand-tune individual fits without re-running everything? That's what I wanted to do with filili. Revisit?
- Ne IX and O VII plot look reversed to each other (red, green, pink crosses are mirrored). IS that physics or am I just lookign at a point cloud?
- Density plot should have right axis, or better background spiterweb with temp, dens (because ratio is also dependent on density somewhat). 
- Improve error estimates using MCMC instead of error propagation. Should also allow priors like "ignore negative N_H region". Some code for that is below, that's not integrated yet. (It's for a different notebook, that I accidentially workedo n separately and that I'm not removing to avoid confusing myself in the future).
- This probes "peak of GofT", so different O VII components are not weighted equally, but those at T near peak are strongest weightied. Check from of GofT for all lines to make sure they are similar. If they are, that's a matter for the discussion section, if they are not, it's a caveat (I don't really havea good idea what to do about it. Maybe the Hegamma lines are visible? THey would provide greater leverage in terms of A_V).

In [ ]:
def fit_triplet_MCMC(obsid, model):
    oids = get_dataids_obsid(obsid)
    for o in oids:
        set_source(o, model)
        set_stat("cash")
    fit(*oids)
    covar(*oids)
    stats, accept, params = get_draws(id=oids[0], otherids=oids[1:], niter=1e4)
    return params

## Reprocess LETGS data in 30 ks chunks so see how many counts we can expect for proposal

See https://cxc.cfa.harvard.edu/cal/Letg/LetgHrcEEFRAC/index.html for how to improve the S/N somewhat with non-standard settings. For the proposal I don't need that, fur a publication it might be worthwhile to explore the difference.

*Note* I don't see changes between chunks, but the wavelength is slightly off (need to fix 0-order location by hand), so I just got the count number from the total 150 ks and devided by five for the proposal.

In [ ]:
# The cells below take a long time to run, so I want ot make sure the notebook stops here.
raise Exception

In [ ]:
import ciao_contrib.runtool as rt

In [ ]:
# Everything could be done with CIAO tools, but I know astropy better, so I migh just as well use that
from astropy.io import fits
from astropy.table import Table

In [ ]:
# From the header for ObsID 6443
# I'm running this on an old computer, thus I don't want to open a large file just to get those two numbers
# when I'm running the notebook again.
TSTART = 272620141.4059900045
TSTOP = 272774659.6507499814

In [ ]:
for i in range(5):
    rt.dmcopy.punlearn()
    rt.dmcopy(infile='data/Chandra/6443/repro/hrcf06443_repro_evt2.fits[EVENTS][time={}:{}]'.format(TSTART + i * delta_t,
                                                                                                    TSTART + (i+1) * delta_t),
              outfile='data/Chandra/6443/repro/evt2_{}'.format(i), option="")
    rt.dmappend(infile='data/Chandra/6443/repro/hrcf06443_repro_evt2.fits[region][subspace -time]',
                outfile='data/Chandra/6443/repro/evt2_{}'.format(i))


In [ ]:
# Assuming DTCOR ppreviously calculated (i.e. for the entire exposure) is OK here, too.
# See https://cxc.cfa.harvard.edu/ciao/threads/spectra_letghrcs/ for how to redo that calculation.
# Not needed for proposal, because effect is in the percent range.

In [ ]:
for i in range(5):
    rt.tgextract.punlearn()
    rt.tgextract(infile='data/Chandra/6443/repro/evt2_{}'.format(i),
                 outfile='data/Chandra/6443/repro/pha2_{}'.format(i),
                 inregion_file='CALDB')

In [ ]:
path = 'data/Chandra/tgcat/obs_6443_tgid_2459/'

for i in range(5):
    ui.load_data('data/Chandra/6443/repro/pha2_{}'.format(i))
    for sign, sherpaid in zip(['-', ''], [1, 2]):
        for num in [1,2,3]:
            ui.load_arf(sherpaid, path+"leg_{}{}.arf.gz".format(sign, num), num)
            ui.load_rmf(sherpaid, path+"leg_{}{}.rmf.gz".format(sign, num), num)
        
    ui.copy_data(1, 't6443_{}_leg-1'.format(i))
    ui.copy_data(2, 't6443_{}_leg+1'.format(i))

In [ ]:
ui.set_analysis('wave')
ui.ignore(None, 13.40)
ui.ignore(13.75, None)
ui.notice(13.40, 13.75)

ui.plot_data('6443_leg-1')
for i in range(5):
    ui.plot_data('t6443_{}_leg-1'.format(i), overplot=True)

In [ ]:
ui.set_analysis('6443_leg-1', "energy", "counts", 0)

In [ ]:
ui.plot_data('6443_leg-1')

In [ ]:
pl = ui.get_data_plot('6443_leg-1')

In [ ]:
pl.y.sum()

In [ ]:
ui.set_analysis('wave')
#ui.ignore(None, 21.5)
#ui.ignore(22.3, None)
#ui.notice(21.5, 22.3)
ui.ignore(None, 13.40)
ui.ignore(13.75, None)
ui.notice(13.40, 13.75)
ui.set_analysis('6443_leg+1', "energy", "counts", 0)
pl = ui.get_data_plot('6443_leg+1')
ui.plot_data('6443_leg+1')
print(pl.y.sum())